In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Read csv files Java Answers

In [2]:
df = dd.read_csv('pmdcodesnippetsviolations_csv/PMDJavaCodeSnippetsViolations*.csv', engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)


## Start a Dask cluster using SLURM jobs as workers

In [3]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-00:30",# walltime="0-00:50",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

Spawn between 20 to 100 workers and connect a client to be able use them.

In [4]:
#cluster.scale(n=20) # ask for 20 jobs or workers
# This also works with adaptive clusters. This automatically launches and kill workers based on load.
# we tell our cluster to autoscale between 10 and 20 workers depending on the load
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
#cluster.adapt(maximum_jobs=20)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.140:37662 Dashboard: http://192.168.94.140:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
df

,beginline,endline,begincolumn,endcolumn,rule,ruleset,class,externalInfoUrl,priority,text
npartitions=204,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [6]:
pmd_codeviolations_init_len = df.shape[0].compute()
print('Initial number of volations {}'.format(pmd_codeviolations_init_len))

Initial number of volations 4896537


In [7]:
# drop empty columns
df = df[~df['class'].isna()] # Drop rows that have NaN in the Code column

In [8]:
pmd_codeviolations_nonan_len =df.shape[0].compute()
print('Number of violations after removing the nans in the class column {}'.format(pmd_codeviolations_nonan_len))

Number of violations after removing the nans in the class column 4887262


In [9]:
print('Difference {}'.format(pmd_codeviolations_init_len - pmd_codeviolations_nonan_len))

Difference 9275


In [10]:
# drop classes that do not have this pattern e.g., Code_10000096_9914015_1959_0
#df = df[df['class'].str.contains('^Code_\d+_\d+_\d+_\d+', regex=True)]
df = df[df['class'].str.contains('^Code_\d+_\d+_\d+_\d+$', regex=True)]

In [11]:
pmd_codeviolations_unkownclass_len = df.shape[0].compute()
print('Number of violations after removing the onces caused by unknown class e.g. inner classes {}'.format(pmd_codeviolations_nonan_len))

Number of violations after removing the onces caused by unknown class e.g. inner classes 4887262


In [12]:
print('Difference {}'.format(pmd_codeviolations_nonan_len - pmd_codeviolations_unkownclass_len))

Difference 499061


#### Count the unique rulesets

In [13]:
#count the unique rulesets
#df.groupby('ruleset').count().shape[0].compute()

#### Get the unique classes dataframe

In [14]:
# get the unique classes
# not the values in the 1st column are where it found the 1st occurances of each unique item
df['ruleset'].drop_duplicates().compute()

0                Design
1            Code Style
11       Best Practices
28          Performance
72          Error Prone
157       Documentation
170      Multithreading
18034          Security
Name: ruleset, dtype: object

In [15]:
ruleset_df = df[['ruleset']]
ruleset_df.head()

,ruleset
0,Design
1,Code Style
2,Code Style
3,Code Style
4,Code Style


In [16]:
# This also counts the unique classes
ruleset_df.index.shape[0].compute()

4388201

### Count the number of times each item occured

In [17]:
#https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
ruleset_count_df = pd.DataFrame(ruleset_df.groupby('ruleset')['ruleset'].count().compute())
# rename the column to total
ruleset_count_df.columns=['total']
# drop the index level
ruleset_count_df = ruleset_count_df.reset_index()
ruleset_count_df

,ruleset,total
0,Best Practices,543884
1,Code Style,2620677
2,Design,762610
3,Documentation,17301
4,Error Prone,319861
5,Multithreading,23796
6,Performance,100010
7,Security,62


### Calculate Percentahe

In [18]:
ruleset_count_df['percent'] = (ruleset_count_df['total']/ ruleset_count_df['total'].sum())*100

In [19]:
ruleset_count_df

,ruleset,total,percent
0,Best Practices,543884,12.394236
1,Code Style,2620677,59.720988
2,Design,762610,17.378648
3,Documentation,17301,0.394262
4,Error Prone,319861,7.289115
5,Multithreading,23796,0.542272
6,Performance,100010,2.279066
7,Security,62,0.001413


### Calculate number to be sampled per ruleset

In [20]:
# https://www.surveysystem.com/sscalc.htm
# put 4887262 into the sample size calculator at 5% confidence interval
ruleset_count_df['sample_size_384'] = round((ruleset_count_df['percent']/ 100)*384, 0)
ruleset_count_df

,ruleset,total,percent,sample_size_384
0,Best Practices,543884,12.394236,48.0
1,Code Style,2620677,59.720988,229.0
2,Design,762610,17.378648,67.0
3,Documentation,17301,0.394262,2.0
4,Error Prone,319861,7.289115,28.0
5,Multithreading,23796,0.542272,2.0
6,Performance,100010,2.279066,9.0
7,Security,62,0.001413,0.0


#### Better computation

In [21]:
def comp_smpl_no(x):
    x = round((x/100)*384)
    if x < 1:
        x = 1
    return x

In [22]:
# https://www.surveysystem.com/sscalc.htm
# put 4887262 into the sample size calculator at 5% confidence interval
import math
ruleset_count_df['sample_size_384'] = ruleset_count_df['percent'].apply(comp_smpl_no)
ruleset_count_df

,ruleset,total,percent,sample_size_384
0,Best Practices,543884,12.394236,48
1,Code Style,2620677,59.720988,229
2,Design,762610,17.378648,67
3,Documentation,17301,0.394262,2
4,Error Prone,319861,7.289115,28
5,Multithreading,23796,0.542272,2
6,Performance,100010,2.279066,9
7,Security,62,0.001413,1
